## 0.00 Setup

In [4]:
%pip install PyMuPDF

   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ----- ---------------------------------- 2.4/16.6 MB 13.4 MB/s eta 0:00:02
   ------------ --------------------------- 5.2/16.6 MB 13.3 MB/s eta 0:00:01
   ------------------ --------------------- 7.9/16.6 MB 12.8 MB/s eta 0:00:01
   ------------------------- -------------- 10.5/16.6 MB 12.8 MB/s eta 0:00:01
   ------------------------------- -------- 13.1/16.6 MB 12.8 MB/s eta 0:00:01
   ------------------------------------- -- 15.7/16.6 MB 12.8 MB/s eta 0:00:01
   ---------------------------------------- 16.6/16.6 MB 12.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Step 0.01 Load libraries and API keys

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv

# Load API keys from .env file 
load_dotenv(override=True)

True

In [ ]:
# Step 0.02: Check API KEY
import os
print(f"[API KEY]\n{os.environ['LANGSMITH_API_KEY']}")

## 1.00 Setup RAG pipeline

In [5]:
### Step 1: Load the documents
loader = PyMuPDFLoader("documents/2025_NARST_conference_program_book.pdf")
docs = loader.load()
print(f"Number of pages in the document: {len(docs)}")

Number of pages in the document: 212


In [6]:
### Step 2 Check the docs
print(docs[3].page_content)

98th NARST International Conference     March 23–26, 2025     4
98th NARST International Conference
General Information
General Information
Information about NARST
NARST is a global organization for improving science 
teaching and learning through research. Since its 
inception in 1928, NARST has promoted research in 
science education and the communication of 
knowledge generated by the research. The ultimate goal 
of NARST is to help all learners achieve science literacy.
The Association is incorporated as a non-profit 
corporation in the State of Minnesota. The official 
publication is the Journal of Research in Science 
Teaching (JRST). NARST encourages presentations of 
a wide variety of investigations in all aspects of science 
education, including action, historical, philosophical, 
ethnographic, experimental, and evaluative research 
studies. Reports of empirical research, critical reviews, 
and theoretical works are encouraged. In October 
2010, to reflect the Association’s gr

In [7]:
### Check metadata
docs[3].__dict__

{'id': None,
 'metadata': {'producer': 'Adobe PDF Library 17.0',
  'creator': 'Adobe InDesign 20.2 (Macintosh)',
  'creationdate': '2025-03-14T13:22:13-04:00',
  'source': 'documents/2025_NARST_conference_program_book.pdf',
  'file_path': 'documents/2025_NARST_conference_program_book.pdf',
  'total_pages': 212,
  'format': 'PDF 1.6',
  'title': '',
  'author': '',
  'subject': '',
  'keywords': '',
  'moddate': '2025-03-17T10:04:33-06:00',
  'trapped': '',
  'modDate': "D:20250317100433-06'00'",
  'creationDate': "D:20250314132213-04'00'",
  'page': 3},
 'page_content': '98th NARST International Conference     March 23–26, 2025     4\n98th NARST International Conference\nGeneral Information\nGeneral Information\nInformation about NARST\nNARST is a global organization for improving science \nteaching and learning through research. Since its \ninception in 1928, NARST has promoted research in \nscience education and the communication of \nknowledge generated by the research. The ultimate